In [ ]:
pip install python-dotenv

In [ ]:
!pip install --upgrade openai httpx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [ ]:
!pip install --upgrade "httpx<0.27.2" # Downgrade httpx to a version less than 0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.2
    Uninstalling httpx-0.27.2:
      Successfully uninstalled httpx-0.27.2


In [ ]:
import os
import json
import datetime
import psutil
from openai import OpenAI
from google.colab import userdata
from typing import Dict, Any, List
from dotenv import load_dotenv

class SystemFunctions:
    @staticmethod
    def get_current_weather(location: str, unit: str = "fahrenheit") -> str:
        """Get the current weather in a given location."""
        weather_data = {
            "tokyo": {"temperature": "10", "unit": unit},
            "san francisco": {"temperature": "72", "unit": unit},
            "paris": {"temperature": "22", "unit": unit}
        }

        location_lower = location.lower()
        if location_lower in weather_data:
            return json.dumps({
                "location": location.title(),
                "temperature": weather_data[location_lower]["temperature"],
                "unit": unit
            })
        return json.dumps({"location": location, "temperature": "unknown"})

    @staticmethod
    def get_current_time_and_date() -> str:
        """Get the current time and date."""
        now = datetime.datetime.now()
        return json.dumps({
            "time": now.strftime("%H:%M"),
            "date": now.strftime("%Y-%m-%d")
        })

    @staticmethod
    def get_battery_level() -> str:
        """Get the current battery level of the system."""
        battery = psutil.sensors_battery()
        if battery:
            return json.dumps({
                "battery_level": battery.percent,
                "is_charging": battery.power_plugged
            })
        return json.dumps({"error": "Battery information not available"})

class OpenAIFunctionCaller:
    def __init__(self):


        # Initialize OpenAI client with minimal configuration
        self.api_key = self._get_api_key()
        self.client = OpenAI(
            api_key=self.api_key,

        )

        self.system_functions = SystemFunctions()
        self.available_functions = {
            "get_current_weather": self.system_functions.get_current_weather,
            "get_current_time_and_date": self.system_functions.get_current_time_and_date,
            "get_battery_level": self.system_functions.get_battery_level
        }

    def _get_api_key(self) -> str:
        """Retrieve OpenAI API key from Google Colab userdata."""
        api_key = userdata.get('OpenAI_api_key')
        if not api_key:
            raise ValueError("OpenAI API key not found. Please set it in your Google Colab environment.")
        return api_key

    def _get_api_key(self) -> str:
        """Retrieve OpenAI API key from Google Colab userdata."""
        load_dotenv('api.env')
        api_key = os.getenv('OPENAI_API_KEY')
        if not api_key:
            raise ValueError("OpenAI API key not found. Please set it in your Google Colab environment.")
        return api_key

    def _get_tools_config(self) -> List[Dict[str, Any]]:
        """Define the available tools configuration."""
        return [
            {
                "type": "function",
                "function": {
                    "name": "get_current_weather",
                    "description": "Get the current weather in a given location.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g., San Francisco, CA.",
                            },
                            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                        },
                        "required": ["location"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_current_time_and_date",
                    "description": "Get the current time and date.",
                    "parameters": {},
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_battery_level",
                    "description": "Get the current battery level of the system.",
                    "parameters": {},
                },
            },
        ]

    def process_tool_calls(self, tool_calls: List[Any], messages: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Process tool calls and append results to messages."""
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = self.available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)

            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
        return messages

    def run_conversation(self, user_message: str = "What's the battery level?") -> str:
        """Run a conversation with the OpenAI model using function calling."""
        try:
            # Initialize conversation
            messages = [{"role": "user", "content": user_message}]

            # First API call with function definitions
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=messages,
                tools=self._get_tools_config(),
                tool_choice="auto"
            )

            response_message = response.choices[0].message
            messages.append(response_message)  # Add model's response to conversation

            # Process tool calls if any
            if response_message.tool_calls:
                messages = self.process_tool_calls(response_message.tool_calls, messages)

                # Second API call with function results
                second_response = self.client.chat.completions.create(
                    model="gpt-4",
                    messages=messages
                )
                return second_response.choices[0].message.content

            return response_message.content

        except Exception as e:
            return f"Error during conversation: {str(e)}"

def main():
    """Main function to demonstrate usage."""
    function_caller = OpenAIFunctionCaller()
    result = function_caller.run_conversation()
    print(result)

if __name__ == "__main__":
    main()

TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [ ]:
import os
import json
import datetime
import psutil
from openai import OpenAI
from google.colab import userdata
from typing import Dict, Any, List

class SystemFunctions:
    @staticmethod
    def get_current_weather(location: str, unit: str = "fahrenheit") -> str:
        """Get the current weather in a given location."""
        weather_data = {
            "tokyo": {"temperature": "10", "unit": unit},
            "san francisco": {"temperature": "72", "unit": unit},
            "paris": {"temperature": "22", "unit": unit}
        }

        location_lower = location.lower()
        if location_lower in weather_data:
            return json.dumps({
                "location": location.title(),
                "temperature": weather_data[location_lower]["temperature"],
                "unit": unit
            })
        return json.dumps({"location": location, "temperature": "unknown"})

    @staticmethod
    def get_current_time_and_date() -> str:
        """Get the current time and date."""
        now = datetime.datetime.now()
        return json.dumps({
            "time": now.strftime("%H:%M"),
            "date": now.strftime("%Y-%m-%d")
        })

    @staticmethod
    def get_battery_level() -> str:
        """Get the current battery level of the system."""
        battery = psutil.sensors_battery()
        if battery:
            return json.dumps({
                "battery_level": battery.percent,
                "is_charging": battery.power_plugged
            })
        return json.dumps({"error": "Battery information not available"})

api_key = userdata.get('OpenAI_api_key')
if not api_key:
  raise ValueError("OpenAI API key not found. Please set it in your Google Colab environment.")

tools = [
            {
                "type": "function",
                "function": {
                    "name": "get_current_weather",
                    "description": "Get the current weather in a given location.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g., San Francisco, CA.",
                            },
                            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                        },
                        "required": ["location"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_current_time_and_date",
                    "description": "Get the current time and date.",
                    "parameters": {},
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_battery_level",
                    "description": "Get the current battery level of the system.",
                    "parameters": {},
                },
            },
  ]

SecretNotFoundError: Secret OpenAI_api_key does not exist.


get current time and date function

get current battery

# Project 1: Tool Calling Chatbot with external APIs connected
Due November 11, 2024 11:59 PM
Instructions
This project involves creating a conversational AI chatbot using the OpenAI API, with integrated tool-calling features to access external libraries and APIs. The chatbot will be designed to respond naturally to user queries, perform various utility tasks, and retrieve real-time information by connecting with at least three key tools.

These tools include:

Datetime for managing date and time-related questions,
Psutil for monitoring system performance metrics like CPU and memory usage,
OpenWeather to provide live weather updates,
NewsAPI for delivering up-to-date news and topic-specific headlines, and
Wolfram Alpha for handling factual and computational queries.

Throughout the development process, it’s encouraged to incorporate AI tools—such as coding assistants and debugging tools—into the workflow to streamline coding and boost productivity. This will provide an opportunity to get comfortable with using AI to enhance development, making it easier to create an intelligent, interactive chatbot that offers practical and timely information.



PLEASE REFERENCE THESE RESOURCES FOR HELP:

AI Midterm Project Document (attached)
The two OpenAI API Notebooks available under the Notebooks folder in the Files section
api_keys_and_chat_completion_api.ipynb
function_calling.ipynb
The OpenAI API Documentation: https://platform.openai.com/docs/api-reference/chat
Class recordings available in the Files section
Slides available in the Files section
Reference materials

AI: An Introduction
Midterm Project
AI Chatbot with Function Calling features
Project Overview:
This project aims to develop a chatbot using the OpenAI API implements
function calling to connect the chatbot to external data sources. Trainees will be
provided a list of libraries and APIs that can be easily attached to the chatbot
to expand its ability.
Libraries and APIs:
● Datetime - Python library that can return the current time and date.
● Psutil - Python library that can return the battery status of the device the
script is running on.
● NewsAPI - News aggregation API that can return the current top
headlines.
● OpenWeatherMap API - Weather API that returns current weather based
on location.
● Wolfram Alpha Short Answers API - Answer engine that returns factual
information.
Project Submission:
Your project will be due on November 7th (11/7). Trainees are given a week to
complete this project.
There will be a Teams assignment created that you will use to submit your
project. You can use one of the following two options to submit your project:
1. You can upload the required files directly to Teams and then submit your
project.
2. You can first upload the required files to a public GitHub repository, then
link to your GitHub repository as your Teams assignment submission.
The required items can be seen below.
AI: An Introduction
Midterm Project
Project Objectives:
1. Create a conversational chatbot using the OpenAI Chat Completions API.
2. Create at least 3 functions that retrieve data from the list of libraries and
APIs provided.
3. Create the tool list that describes these functions and the arguments
necessary for the function in natural language.
4. Append the data returned from the called function to the message list
with the role “tool” and make another API call for the final response.
5. Document the entire process for reproducibility and future reference.
Project Tasks:
1. Conversational Chatbot
a. Make a loop where the prompt and response pairs are appended to
the message list.
2. Data Retrieval Functions
a. Set up accounts and API keys for the chosen data source APIs.
b. Use the python libraries and APIs within functions to return the
desired data.
3. Function Description
a. Detail the intended use of the functions in natural language so the
model can understand when to use the functions properly.
b. Determine what arguments (if any) are necessary for each function.
4. Function Calling Implementation
a. Use the response object to call the function(s) chosen by the model.
b. Append the function result to the message list for the model to
generate the natural language response.
5. Documentation
a. Create detailed documentation explaining the generation steps,
assumptions, and decisions.
b. Include comments in the code to make it more understandable and
maintainable.
6. Testing and Validation
AI: An Introduction
Midterm Project
a. Test the function calling with multiple prompts that invoke different
(or multiple) functions to be called.
b. Validate the results against the expected outcomes.
c. BONUS: Automate the testing using a dictionary of prompts and
expected function calls.
Items for Submission:
1. A documented generation process, including code comments and
explanations.
2. A script for the conversational function calling chatbot.
3. Any additional documentation or instructions for using the chatbot.
AI: An Introduction
Midterm Project
Rubric:
If you do not score a 12 or higher for your project based on the rubric below you will have the
opportunity to make changes to your project and resubmit it.
0 Points -
Missing
1 Point - Does
Not Meet
Expectations
2 Points - Meets
Expectations
3 Points -
Exceeds
Expectations
Points Awarded
Conversational
Chatbot
This chatbot only
responds to one
prompt and
doesn’t append
the response to
the message list.
This chatbot
doesn’t loop
which restricts
users from
prompting
multiple times
when running
the script.
This chatbot
appends user,
assistant, and
tool messages to
the message list
in a loop for
users to prompt
the chatbot
multiple times
within a run.
This chatbot
manages the
message list
properly within a
loop to allow for
multiple uses
when running
the script
without hitting
the context limit.
Data Retrieval
Functions
Functions didn’t
use APIs or
python libraries
to retrieve the
desired data.
Only some
functions
performed the
proper data
retrieval.
APIs and python
libraries were
used properly to
access data and
return it to the
model.
APIs and python
libraries were
used in a
multitude of
ways to access
different sets of
AI: An Introduction
Midterm Project
data to return to
the model.
Function
Description
No function
descriptions
were provided.
Some function
descriptions are
vague and
unclear about
the actions the
function take
and the
arguments
necessary.
Function
descriptions are
easily
understood and
detailed with
proper argument
details.
Function
descriptions are
easily
understood and
detailed with
proper argument
details, required
arguments, and
any necessary
units.
Function Calling
Implementation
The function(s)
listed in the
response object
weren’t called in
the script
properly.
The function
results are
displayed to the
user but not
appended to the
message list for
the model’s
informed
response.
The function
results are
appended to the
message list
properly.
Parallel function
calling is
supported and
the function
results are
appended to the
message list
properly.
Documentation No
documentation
Only some of the
needed
Code was
written in the
All parts of the
code were
AI: An Introduction
Midterm Project
was added. documentation
was added.
proper order of
events and is
well
documented.
written in the
proper order of
events, and a
general process
was written for a
non-technical
user.
Testing and
Validation
No testing or
validations were
performed.
Only some
testing and
validation were
done, but not
enough to catch
all errors or
issues.
Adequate testing
was performed,
and the output
was validated
against
expected
responses.
A dictionary of
prompts and
expected
responses were
created to
automated
testing and
validation.
Score